# Walmart Recruiting - Store Sales Forecasting

* ***Objetivo:*** Prever o volume de vendas semanal das lojas Walmart
* ***Relevância do Problema:*** Ser capaz de estimar as vendas em cada loja pode trazer benefícios significativos nos planejamentos de estoque e logística, alem de alocação de funcionários e gestão de recursos e campanhas de marketing.
* ***Importância dos Feriados no Problema:*** Todos os aspectos citados anteriormente podem ser impactados significativamente por datas que influenciam de maneira extraordinária no volume de vendas.

# Instalando pacotes adicionais necessários

In [ ]:
!pip install scikit-optimize --quiet

# Carregando Bibliotecas necessárias para o desenvolvimento do trabalho

In [ ]:
import os
from datetime import datetime
import math
import warnings
from zipfile import ZipFile

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV


from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import pickle

# Upload dados

Nesta etapa deixei duas versões possíveis, uma para o upload dos arquivos dentro do kaggle e outra para o uso na máquina local.

## Upload dentro do Kaggle

In [ ]:
# Folder com as bases disponíveis
os.listdir('../input/walmart-recruiting-store-sales-forecasting')

In [ ]:
# Descompactando os arquivos

with ZipFile('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip') as f:
    f.extractall(path='walmart-recruiting-store-sales-forecasting')

with ZipFile('../input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip') as f:
    f.extractall(path='walmart-recruiting-store-sales-forecasting')
    
with ZipFile('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip') as f:
    f.extractall(path='walmart-recruiting-store-sales-forecasting')

with ZipFile('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip') as f:
    f.extractall(path='walmart-recruiting-store-sales-forecasting')

os.listdir('walmart-recruiting-store-sales-forecasting')

In [ ]:
# Carregando as bases em dataframes panda
features = pd.read_csv("./walmart-recruiting-store-sales-forecasting/features.csv")
stores = pd.read_csv("../input/walmart-recruiting-store-sales-forecasting/stores.csv")
train = pd.read_csv("./walmart-recruiting-store-sales-forecasting/train.csv")
test = pd.read_csv("./walmart-recruiting-store-sales-forecasting/test.csv")
submission = pd.read_csv("./walmart-recruiting-store-sales-forecasting/sampleSubmission.csv")

## Acessando arquivos via Google Colab ou na máquina local

In [ ]:
# drive.mount('/content/gdrive')

In [ ]:
# Definindo caminho para os arquivos:
# 'gdrive/My Drive/wallmart_sales_forecasting/'

# features = pd.read_csv('gdrive/My Drive/wallmart_sales_forecasting/features.csv')
# stores = pd.read_csv('gdrive/My Drive/wallmart_sales_forecasting/stores.csv')
# train = pd.read_csv('gdrive/My Drive/wallmart_sales_forecasting/train.csv')
# test = pd.read_csv('gdrive/My Drive/wallmart_sales_forecasting/test.csv')
# submission = pd.read_csv('gdrive/My Drive/wallmart_sales_forecasting/submission.csv')

In [ ]:
print("features.shape", features.shape)
print("stores.shape", stores.shape)
print("train.shape", train.shape)
print("test.shape", test.shape)
print("submission.shape", submission.shape)

## Visualizando os dataframes

Foram disponibilizadas bases separadas, com informações distribuidas entre elas. No total são 5 arquivos diferentes.
* **Features:** Contem as variáveis explicativas que deve ajudar no desenvolvimento do modelo;
* **Stores:** Contém a informação das lojas em relação ao tipo e ao tamanho (Size);
* **Train:** Traz as informações de venda por loja, departamento e data de referência, além da informação de feriados, que parece ser bem relevante pelo enunciado do problema;
* **Test:** Contém as mesmas informações disponibilizadas na base de treino, mas para datas diferentes e;
* **Submission:** Contém a base para ser escorada e submetida para avaliação na competição

### Stores
* Traz os campos 'Store', com valores variando de 1 a 45, e 'Type', com valores entre 1 e 3

In [ ]:
stores.head()

In [ ]:
stores.Store.unique()

In [ ]:
stores.Type.unique()

### Train
* Contém os campos:
* **'Store'** e **'Depto'**, que ajudam a identificar o volume de vendas por loja e por departamento;
* **'Date'** e **'Weekly_Sales'**, que trazer a combinação entre semana e volume de vendas naquela semana, respectivamente e;
* **'IsHoliday'**, que diz se existe um feriado naquela semana.

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
submission.head()

In [ ]:
features.head()

# Criação da Base inicial

Aqui é importante agrupar as informações em uma única base para facilitar a preparacao e o tratamento necessário das variáveis.

In [ ]:
df_data = train.merge(features
                       ,on = ['Store','Date','IsHoliday']
                       ,how = 'inner').merge(stores
                                            ,on = ['Store']
                                            ,how = 'inner')

In [ ]:
df_data.head()

# Análise Exploratória

* O Objetivo desta etapa é entender o dataset um pouco mais a fundo.
* O primeiro passo é entender o evento, como está suaa distribuição em relação ao tempo, se existe sazonalidade e em que granularidade.
* Como o enunciado dá bastante ênfase aos feriados, podemos partir deste parâmetro, e depois podemos aprofundar as análises.

## Variação do volume de vendas em relação aos feriados

In [ ]:
plt.figure()
plt.title ('Volume de vendas semanais em função dos feriados')
fig = sns.boxplot(x = 'IsHoliday'
                  ,y = 'Weekly_Sales'
                  ,data = df_data[['Weekly_Sales','IsHoliday']]
                  ,showfliers = False)

* Quando comparamos apenas o volume de vendas em relação aos feriados, não encontramos grande diferença.
* Deste dado, podemos supor que nem todos os feriados influenciam no volume de vendas, ou que a própria variacao do volume de vendas é muito grande.

### Avaliação do volume de vendas por mês

* Como não encontramos uma diferença significativa entre 'Feriados' e 'Não Feriados', decidi investigar a relacao entre feriados e vendas um pouco mais a fundo.
* Aqui podemos avaliar o volume de vendas por mês, com os meses que possuem feriados destacados

In [ ]:
# Função para expandir o campo data em intervalos que podem ser relevantes no futuro

def split_date(df,date):

    '''
    Transforma o campo indicado na entrada em data e extrai valores que podem ser relevantes, como ano, mês e dia. 
    A saida da função concatena esses novos campos no dataframe indicado na entrada
    '''
    
    df['dt_ref'] = pd.to_datetime(df[date])
    df['year'] = df.dt_ref.dt.year
    df['month'] = df.dt_ref.dt.month
    df['day'] = df.dt_ref.dt.day
    df['week_of_year'] = df.dt_ref.dt.isocalendar().week
    df['period_month'] = df_data.dt_ref.dt.to_period('M')

In [ ]:
split_date(df_data, 'Date')

# df_data[['month','Weekly_Sales','IsHoliday']].head()

plt.figure(figsize = (20,6))
plt.title ('Variação do volume de vendas em relação aos meses do ano')
fig = sns.boxplot(x = 'month'
                  ,y = 'Weekly_Sales'
                  ,data = df_data[['month','Weekly_Sales','IsHoliday']]
                  ,showfliers = False
                  ,hue = 'IsHoliday')

* Quando avaliamos o volume de vendas mês a mês, percebemos que os feriados influenciam sim nas vendas, mas que essa influência é diferente entre os feriados. 

### Variação do volume de vendas por loja e influência dos feriados nas vendas de cada loka

* Com essa visualização podemos investigar a variação do volume de vendas entre as lojas e também o quanto os feriados impactam as vendas em cada loja.

In [ ]:
plt.figure(figsize = (20,6))
plt.title ('Variação do volume de vendas em relação às lojas e impacto dos feriados')
fig = sns.boxplot(x = 'Store'
                  ,y = 'Weekly_Sales'
                  ,data = df_data[['Store','Weekly_Sales','IsHoliday']]
                  ,showfliers = False
                  ,hue = 'IsHoliday')

* Aqui podemos perceber que o volume de vendas varia muito entre as lojas. No entanto, ainda não é possível perceber claramente o impacto dos feriados nas vendas.

### Variação do volume de vendas entre os departamentos e influência dos feriados 

* Aqui repetimos a análise realizada por loja, mas olhando para os departamentos.

In [ ]:
plt.figure(figsize = (20,8))
plt.title ('Variação do volume de vendas em relação aos departamentos e impacto dos feriados', fontsize=16)
fig = sns.boxplot(x = 'Dept'
                  ,y = 'Weekly_Sales'
                  ,data = df_data[['Dept','Weekly_Sales','IsHoliday']]
                  ,showfliers = False
                  ,hue = 'IsHoliday')

* Aqui é possível notar o impacto do feriado em departamentos específicos, como o 55 e o 72.
* No entanto, também é possivel observar influência negativa do feriado no departamento 65.

### Volume de vendas por semana ao longo do ano

* Por fim, podemos avaliar o volume de vendas por semana, se existe alguma sazonalidade e/ou influência dos feriados que possa nos direcionar na construção de novas variáveis e no desenvolvimento do  modelo.

In [ ]:
weekly_sales_2010 = df_data[df_data.year==2010].groupby('week_of_year')['Weekly_Sales'].mean()
weekly_sales_2011 = df_data[df_data.year==2011].groupby('week_of_year')['Weekly_Sales'].mean()
weekly_sales_2012 = df_data[df_data.year==2012].groupby('week_of_year')['Weekly_Sales'].mean()

plt.figure(figsize=(20,6))
plt.plot(weekly_sales_2010.index, weekly_sales_2010.values)
plt.plot(weekly_sales_2011.index, weekly_sales_2011.values)
plt.plot(weekly_sales_2012.index, weekly_sales_2012.values)

plt.xticks(np.arange(1, 53, step=1), fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('Week of Year', fontsize=16, labelpad=20)
plt.ylabel('Sales', fontsize=20, labelpad=20)

plt.title("Volume de vendas por semana, por ano", fontsize=16)
plt.legend(['2010', '2011', '2012'], fontsize=14);

* De forma similar ao que observamos na análise do volume de vendas por mês, podemos notar um aumento expressivo no número de vendas nas semanas referentes aos feriados de ação de graças e natal.
* Este ponto indica que talvez seja importante criar variáveis adicionais para separar os feriados, como uma combinação de período do ano por exemplo.

# Análise Exploratória das variáveis explicativas

* Após entender um pouco melhor o comportamento das vendas, é importante entender e avaliar as variáveis que podem ajudar na predição do volume de vendas.
* O primeiro passo é dividir as variáveis em numéricas e categóricas.
* Outro ponto imporante é a consistência desses campos, que pode influenciar consideravelmente na estabilidade do modelo ao longo do tempo.



### Concentração de informações faltantes 'Missings'

In [ ]:
df_data.isna().sum()

Como citado no enunciado do problema, as informações dos campos 'markdown' só estão disponíveis a partir de um período específico.

In [ ]:
df_data.tail()

## Separando as variáveis em numéricas e categóricas

In [ ]:
df_data.columns

In [ ]:
# Variáveis para o modelo
input_cols = ['Store', 'Dept', 'Weekly_Sales', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'year',
       'month', 'day', 'week_of_year']

# Variáveis numéricas
var_num = ['Store', 'Dept', 'Size','Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'year', 'month', 'day',
       'week_of_year']

# Variáveis categóricas
var_cat = ['IsHoliday', 'Type']

# Variável alvo
target_col = 'Weekly_Sales'

### Função para consolidar as principais informações referentes às variáveis

In [ ]:
def Metadados(dataframe, 
              target_var, 
              id_vars = [], 
              vars_quanti = [], 
              vars_quali = [], 
              vars_bin = []): 
    '''
    Cria um novo df com pricipais caracteristicas do df inicial,
    como tipo de variaveis, quantidade de valores diferentes, 
    quantidade de missings, etc.
    
    Usa como entrada o df original, a variavel alvo, as vars utilizadas como ID, 
    a lista de variaveis quantitativas, ou continuas e as variaveis qualitativas, 
    binarias ou nao
    '''

    
    '''
    
    Entradas: DF original
              var resposta
              vars ID
              lista de vars quanti
              lista de vars quali
              lista de vars binárias
              
    Saida:    DF com proprieades do DF original
    
    '''
    
    train = dataframe

    missing = []
    data = []
    dtype = []
    card = []
    lista_mod = []
    
    for f in train.columns:

        missing=train[f].isnull().sum()
        
        # Definindo o papel das variáveis:
        if f == target_var: #Variável Resposta
            role = 'target'
        elif f in id_vars:
            role = 'id'          
        else:
            role = 'input'

        # Definindo o tipo das variáveis da tabela de entrada
        dtype = train[f].dtype
        
        # Quantidade de domínios distintos para cada variável do tipo ordinal e nominal
        card = train[f].value_counts().shape[0]
        
        # Definindo lista que a var se encaixa('quanti' ou 'quali')
        if f in list(vars_quanti):
            lista_mod = 'quanti'
        elif f in list(vars_quali):
            lista_mod= 'quali'
        else: 
            lista_mod = 'outra'

        # Criando a lista com todo metadados
        f_dict = {'Variaveis': f
                  ,'Role': role
                  ,'Type': dtype
                  ,'Missing': missing
                  ,'Cardinalidade': card
                  ,'Lista_mod': lista_mod}
        data.append(f_dict)

    meta = pd.DataFrame(data, columns=['Variaveis'
                                       ,'Role'
                                       ,'Type'
                                       ,'Missing'
                                       ,'Cardinalidade'
                                       ,'Lista_mod'])


    return meta 

In [ ]:
Meta_raw = Metadados(df_data, target_col, vars_quanti = var_num, vars_quali = var_cat)

In [ ]:
Meta_raw

### Input de missings

* Aqui tomei a decisão de substituir os valores faltantes por 0, indicando que a falta do registro poderá ser entendida como ausência de ações promocionais.

In [ ]:
df_data['MarkDown1'].fillna(0, inplace=True)
df_data['MarkDown2'].fillna(0, inplace=True)
df_data['MarkDown3'].fillna(0, inplace=True)
df_data['MarkDown4'].fillna(0, inplace=True)
df_data['MarkDown5'].fillna(0, inplace=True)

# Variáveis Numéricas ou Contínuas

* Aqui as variáveis numéricas foram 'quebradas' em faixas de valor e comparadas com o volume de vendas.
* O Objetivo aqui é avaliar o comportamento das variáveis em relação ao evento.
* Existem algumas formas diferentes de realizar essa análise, mas desta forma conseguimos avaliar se existe uma relação linear entre as variáveis e o evento, se existe um pico de vendas relacionado com alguma faixa de alguma variável, ou se alguns campos parecem não trazer informações relevantes para o modelo.

In [ ]:
df_var_num = df_data[['Weekly_Sales']]

df_bivar = pd.DataFrame()
aux_concent_num = pd.DataFrame()
aux_drop_num = []

for i in var_num:
    
#         print (aux_var)
    aux_var = 'cat_' + i
    df_var_num[aux_var] = pd.qcut(df_data[i], 10,labels=False, duplicates = 'drop')
    
    if df_data[i].nunique() < 2:
        aux_drop_num.append(i)

    else:
        max_concent = df_var_num[aux_var].value_counts(normalize = True).max()
        aux_df = pd.DataFrame({'vars':[i]
                               ,'concentracao':max_concent})
        aux_concent_num = pd.concat([aux_concent_num,aux_df], axis = 0)
    
        aux_bivar = pd.DataFrame()
        aux_bivar['Weekly_Sales'] = df_var_num[[aux_var,'Weekly_Sales']].groupby([aux_var]).mean().Weekly_Sales    
        aux_bivar['pop'] = df_var_num[[aux_var]].groupby(df_var_num[aux_var]).count()
        aux_bivar.loc[:,'taxa_evento'] = (aux_bivar.loc[:,'Weekly_Sales'] / aux_bivar.loc[:,'pop'])*100
        aux_bivar['taxa_med'] = (aux_bivar.Weekly_Sales.sum()/aux_bivar['pop'].sum())*100
        aux_bivar['var'] = i
        aux_bivar['var_cat']= aux_var
        aux_bivar['var_tipo'] = 'quanti'
        aux_bivar.reset_index(drop = False, inplace = True)

        aux_bivar.columns = ['cat'
                            ,'evento'
                            ,'pop'
                            ,'taxa_evento'
                            ,'taxa_med' 
                            ,'var'
                            ,'var_cat'
                            ,'var_tipo']

        #print()
        #print(aux_bivar)
        #print()


        df_bivar = pd.concat([df_bivar,aux_bivar], axis = 0)

        # Cria a janela
        fig, ax1 = plt.subplots()

        # Taxa de evento por categorias
        color = 'tab:red'
        plt.title(aux_var)
        ax1.set_xlabel('faixas')
        ax1.set_ylabel('Weekly Sales', color=color)
        ax1.scatter(aux_bivar['cat'], aux_bivar['taxa_evento'], color=color, label = 'taxa evento')
        ax1.tick_params(axis='y', labelcolor=color)



        #ylim
        if aux_bivar['taxa_evento'].min() < 10:
            bottom = 0
        else:
            bottom = round(aux_bivar['taxa_evento'].min() - 10)

        if aux_bivar['taxa_evento'].max() > 90:
            up = 100
        else:
            up = round(aux_bivar['taxa_evento'].max() + 10)

        plt.ylim(bottom,up)


        for x,y in zip(aux_bivar['cat'], aux_bivar['taxa_evento']):

            label = "{:.2f}".format(y)

            # aqui incluimos o rotulo para cada ponto dos graficos
            plt.annotate(label, # texto
                         (x,y), # ponto
                         textcoords="offset points", # posicao do rotulo
                         xytext=(0,10), # distancia do texto ao ponto (x,y)
                         ha='center') # 'horizontal alignment' pode ser left, right or center

        # Taxa de evento média
        color = 'tab:green'
        ax1.plot(aux_bivar['cat'], aux_bivar['taxa_med'], color=color, linestyle='dashed', label='taxa media de evento')

        # Publico em cada categoria
        ax2 = ax1.twinx()  # segundo eixo instanciado, com o primeiro 
        color = 'tab:grey'
        ax2.set_ylabel('pop', color=color)  
        ax2.bar(aux_bivar['cat'], aux_bivar['pop'], color=color, alpha = 0.2, label = 'pulico por categoria')
        ax2.tick_params(axis='y', labelcolor=color)

        #fig.tight_layout()  # otherwise the right y-label is slightly clipped
        plt.show()            
            
aux_concent_num = aux_concent_num.reset_index(drop=True)

* Podemos observar um comportamento linear em relacao ao evento para algumas variáveis, como o tamanho das lojas (Size)
* Outras variáveis parecem não estar em nada associadas ao volume de vendas, como o preço do combustível.
* De forma geral, as variáveis não demonstram um comportamento linear em relação ao volume de vendas.
* Isso indica que, provavelmente, um modelo não linear será a melhor opção para resolver esse problema.
* Outra possivel solução seria utilizar essas variáveis divididas em faixas, agrupando valores de acordo com a relação com o volume de vendas.


In [ ]:
aux_drop_num

In [ ]:
aux_concent_num

## Correlação entre Variáveis Numéricas

Embora a correlação não impacte muito os modelos não-lineares, é importante ter conhecimento do comportamento e da correlação entre as variáveis para conseguir avaliar sua importância no modelo final.

In [ ]:
plt.figure(figsize=(16, 16))

corr = df_data[var_num].corr(method = 'pearson')
mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(corr, mask = mask,annot = True)

* Dentre as variáveis analisadas, chamou atenção MarkDown4 e 1, além do ano com o preco do preço do combustível.


# Variáveis Categóricas

* **Seguindo o mesmo processo para as variáveis categóricas**

In [ ]:
var_cat

In [ ]:
df_var_cat = df_data[['Weekly_Sales']]

aux_concent_cat = pd.DataFrame()
aux_drop_cat = []

for i in var_cat:
    
    aux_var = 'cat_' + i
    df_var_cat[aux_var] = df_data[i]
    if df_var_cat[aux_var].nunique() < 2:
        aux_drop_cat.append(i)

    else:
        max_concent = df_var_cat[aux_var].value_counts(normalize = True).max()
        aux_df = pd.DataFrame({'vars':[i]
                               ,'concentracao':max_concent})
        aux_concent_cat = pd.concat([aux_concent_cat,aux_df], axis = 0)    
    
        
        aux_bivar = pd.DataFrame()
    #print (aux_var)
    
    

        aux_bivar['Weekly_Sales'] = df_var_cat[[aux_var,'Weekly_Sales']].groupby([aux_var]).mean().Weekly_Sales
        aux_bivar['pop'] = df_var_cat[[aux_var]].groupby(df_var_cat[aux_var]).count()
        aux_bivar.loc[:,'taxa_evento'] = (aux_bivar.loc[:,'Weekly_Sales'] / aux_bivar.loc[:,'pop'])*100
        aux_bivar['taxa_med'] = (aux_bivar.Weekly_Sales.sum()/aux_bivar['pop'].sum())*100
        aux_bivar['var'] = i
        aux_bivar['var_cat']= aux_var
        aux_bivar['var_tipo'] = 'quali'
        aux_bivar.reset_index(drop = False, inplace = True)

        aux_bivar.columns = ['cat'
                            ,'evento'
                            ,'pop'
                            ,'taxa_evento'
                            ,'taxa_med' 
                            ,'var'
                            ,'var_cat'
                            ,'var_tipo']

        #print()
        #print(aux_bivar)
        #print()


        df_bivar = pd.concat([df_bivar,aux_bivar], axis = 0)


        # Cria a janela
        fig, ax1 = plt.subplots()

        # Taxa de evento por categorias
        color = 'tab:red'
        plt.title(aux_var)
        ax1.set_xlabel('categorias')
        ax1.set_ylabel('Weekly Sales', color=color)
        ax1.scatter(aux_bivar['cat'], aux_bivar['taxa_evento'], color=color, label = 'taxa evento')
        ax1.tick_params(axis='y', labelcolor=color)

        #ylim
        if aux_bivar['taxa_evento'].min() < 10:
            bottom = 0
        else:
            bottom = round(aux_bivar['taxa_evento'].min() - 10)

        if aux_bivar['taxa_evento'].max() > 90:
            up = 100
        else:
            up = round(aux_bivar['taxa_evento'].max() + 10)

        plt.ylim(bottom,up)

        for x,y in zip(aux_bivar['cat'], aux_bivar['taxa_evento']):

            label = "{:.2f}".format(y)

            # aqui incluimos o rotulo para cada ponto dos graficos
            plt.annotate(label, # texto
                         (x,y), # ponto
                         textcoords="offset points", # posicao do rotulo
                         xytext=(0,10), # distancia do texto ao ponto (x,y)
                         ha='center') # 'horizontal alignment' pode ser left, right or center

        # Taxa de evento média
        color = 'tab:green'
        ax1.plot(aux_bivar['cat'], aux_bivar['taxa_med'], color=color, linestyle='dashed', label='taxa media de evento')

        # Publico em cada categoria
        ax2 = ax1.twinx()  # segundo eixo instanciado, com o primeiro 
        color = 'tab:grey'
        ax2.set_ylabel('pop', color=color)  
        ax2.bar(aux_bivar['cat'], aux_bivar['pop'], color=color, alpha = 0.2, label = 'pulico por categoria')
        ax2.tick_params(axis='y', labelcolor=color)

        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        plt.show()
        
aux_concent_cat = aux_concent_cat.reset_index(drop=True)

In [ ]:
aux_drop_cat

In [ ]:
aux_concent_cat


* No caso das variáveis categóricas, podemos observar valores médios de vendas bem distintos entre as categorias, o que indica que essas variáveis podem ser relevantes para o modelo.

## Feature Engeneering

* Aqui podemos utilizar um valor relativo para algumas variáveis contínuas. 
* Por exemplo, podemos comparar o valor da Temperatura na semana em questão com o valor médio do mês, dos ùltimos 3, 6 ou até 12 meses. Desta forma conseguimos avaliar se a mudança na temperatura em relação a um período anterior impacta o volume de vendas.
* O racional é o mesmo para todos os campos aqui.

In [ ]:
df_data[input_cols].head()

In [ ]:
mean_list = ['Temperature','Fuel_Price','CPI','Unemployment']

In [ ]:
def variaveis_relativas(df, lista_vars, var_group):
    
    '''
    O objetivo desta função é criar 'valores relativos', que indicam o quanto o valor mais atual de distancia do valor médio de um período de interesse.
    Aqui foi testado o valor da variável em relação à média mensal, que é indicada pela variável 'var_group'.
    Entradas: DataFrame com todas as informações necessárias para calcular os valores consolidados, lista dos campos que serão transformados e a variável sobre a qual os valores serão agrupados
    Saídas: A função devolve o mesmo DataFrame, com as novas variáveis concatenadas 
    '''
    
    # Calculando valores médios para servir de referencial 
    aux_mean = pd.DataFrame()
    aux_names = [var_group]
    
    for var in lista_vars:
        #print(var)
        aux_mean[var] = df[[var_group,var]].groupby([var_group]).mean()
        aux_name_2 = 'mean_' + var
        aux_names.append(aux_name_2)

    aux_mean.reset_index(inplace = True)

    # Ajustando o nome dos campos
    aux_mean.columns = aux_names

    df = df.merge(aux_mean
                  ,on = [var_group]
                  ,how = 'inner')
    
    #Incluindo as variáveis no dataframe
    for var in lista_vars:
        var_name = 'relative_' + var
        mean_var = 'mean_' + var
        df[var_name] = (df[var] + 0.0001)/df[mean_var]
        
    return df

In [ ]:
df_train = variaveis_relativas(df_data, mean_list, 'month')

In [ ]:
df_train.head()

In [ ]:
df_train.columns

In [ ]:
# Incluindo as novas variáveis na lista
var_num_2 = var_num + ['relative_Temperature'
                       ,'relative_Fuel_Price'
                       ,'relative_CPI'
                       ,'relative_Unemployment']

## Correlação entre Variáveis Numéricas - Incluindo novas variáveis

Mesmo processo realizado anteriormente, mas adicionando as novas variáveis que, possivelmente, estarão correlacionadas com suas origens (valores absolutos)

In [ ]:
plt.figure(figsize=(16, 16))

corr = df_train[var_num_2].corr(method = 'pearson')
mask = np.triu(np.ones_like(corr, dtype=bool))

sns.heatmap(corr, mask = mask,annot = True)

* Como esperado, com exceção da temperatura, as variáveis construídas são altamente correlacionadas com as variáveisl originais.
* Se formos trabalhar com um modelo de regressão, é importante tratar de forma específica estas variáveis. Como os modelo não lineares não são tão impactados por variáveis correlacionadas, podemos seguir coma s variáveis e depois fazemos a escolha da variável que resultar em um valor de importância maior e aplicamos este conceito para os modelos lineares também.
* Adicionamentel, este ponto poderia ser minimizado aumentando o períodp médio de comparação, antes de construir o valor relavito à média.
* Isso pode ser testado para uma segunda versão do modelo final.


# Incluindo as variáveis categóricas no DataFrame normalizado

In [ ]:
# Binarizando as variáveis categóricas para evitar lidar com campos de texto
df_train = pd.get_dummies(df_train, columns = var_cat)

In [ ]:
df_train.head()

# Divisão da base em treino, teste e validação

* Aqui dividimos a base em treino e teste, antes de utilizar a base 'test' de fato.
* Além disso, podemos separar uma data de referência mais recente para avaliar o modelo em um intervalo de tempo em que ele ainda não foi treinado. 
* Desta forma minimizamos a chance de overfit e podemos avaliar a estabilidade do modelo ao longo do tempo.

In [ ]:
df_train.period_month.unique()

In [ ]:
# Selecionando mês mais recente na base para validação
df_val = df_train[df_train.period_month == '2012-10'].copy()

df_train = df_train[df_train.period_month != '2012-10']

In [ ]:
# Inclusão das variáveis categóricas, já binarizadas, na lista geral de variáveis.
vars_model = var_num_2 + ['IsHoliday_True', 'Type_A', 'Type_B', 'Type_C']
df_train[vars_model].head()

In [ ]:
# Dividindo a base de treino em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df_train.loc[:,df_train.columns != 'Weekly_Sales'], # Variáveis Explicativas
                                                    df_train.Weekly_Sales,  # Variável Resposta
                                                    test_size = 0.3, # Proporção entre treino e teste
                                                    random_state = 0)

# Normalização das variáveis numéricas

* Embora os modelos não lineares não sejam sensíveis à diferenças de amplitude entre as variáveis, a ideia é testar diversas técnicas diferentes. 

In [ ]:
# Salvando as informaçções originais para análises futuras
viz_result_test = X_test[['Store', 'Dept', 'Date', 'year', 'week_of_year']].copy()
viz_result_valida = df_val[['Store', 'Dept', 'Date', 'year','week_of_year']].copy()

In [ ]:
viz_result_test.head()

In [ ]:
scaler = StandardScaler().fit(X_train[var_num_2])
X_train[var_num_2] = scaler.transform(X_train[var_num_2])
X_test[var_num_2] = scaler.transform(X_test[var_num_2])
df_val[var_num_2] = scaler.transform(df_val[var_num_2])

## Salvando Scaler para utilizar em produção

In [ ]:
scaler_Pkl = pickle.dumps(scaler)

In [ ]:
X_train[var_num_2].head()

# Treinando os Modelos

In [ ]:
def ml_error(model_name, y, yhat):
    '''
    Esta função auxilia na avaliação dos modelos utilizando as métricas escolhidas
    Entradas: Nome dado ao modelo, valores reais e valores preditos 
    Saídas: Dataframe com nome do modelo, erro médio absoluto e erro médio quadrado
    '''
    mae = mean_absolute_error(y, yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))
    
    return pd.DataFrame({'Model Name' : model_name,
                        'MAE' : mae,
                        'RMSE' : rmse}, index = [0])

## Baseline - Regressão Linear

In [ ]:
# DataFrame para guardar a informação sobre o desempenho dos modelos
aux_compara = pd.DataFrame()

In [ ]:
LR = LinearRegression()

LR.fit(X_train[vars_model],y_train)


In [ ]:
y_pred = LR.predict(X_test[vars_model])
y_val = LR.predict(df_val[vars_model])

#performance oos
lr_result_oos = ml_error('Linear Regression OOS', y_test, y_pred)
lr_result_oos

In [ ]:
# performance oot
lr_result_oot = ml_error('Linear Regression OOT', df_val.Weekly_Sales, y_val)
lr_result_oot

In [ ]:
aux_compara = pd.concat([aux_compara, lr_result_oos],axis = 0)
aux_compara = pd.concat([aux_compara, lr_result_oot],axis = 0)


aux_compara

## Regressão Linear - l1 (Lasso)

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'alpha':[0.01, 0.1, 0.5, 2, 5]}

# model
lrr = Lasso()
lrr_clf = GridSearchCV(lrr, parameters)
lrr_clf.fit(X_train[vars_model], y_train)

In [ ]:
lrr_clf.best_params_

In [ ]:
# prediction
y_pred_lrr = lrr_clf.predict(X_test[vars_model])
y_val_lrr = lrr_clf.predict(df_val[vars_model])

#performance
lrr_result_oos = ml_error('Linear Regression - Lasso OOS', y_test, y_pred_lrr)
lrr_result_oos

In [ ]:
lrr_result_oot = ml_error('Linear Regression - Lasso OOT', df_val.Weekly_Sales, y_val_lrr)
lrr_result_oot

In [ ]:
aux_compara = pd.concat([aux_compara, lrr_result_oos],axis = 0)
aux_compara = pd.concat([aux_compara, lrr_result_oot],axis = 0)


aux_compara

## Random Forrest

* Como o tempo de execução dos notebooks na plataforma é limitada, acabei rodando essa etapa do GridSearchCV fora deste notebok.
* De qualquer forma, mantive aqui o código, como ele foi utilizado, porém comentado. 

In [ ]:
# parameters = {'n_estimators': [100, 250, 500]
              # ,'max_depth': [None, 3, 4, 5]
              # ,'max_features': [0.25, 0.5, 0.75]}

# model
# rf = RandomForestRegressor()
# rf_reg = GridSearchCV(rf, parameters)
# rf_reg.fit(X_train[vars_model], y_train)

# rf_reg.best_params_

In [ ]:
# model
rf = RandomForestRegressor(n_estimators = 100
                           ,max_depth = None
                           ,max_features = 0.75)

rf.fit(X_train[vars_model], y_train)

In [ ]:
# prediction
y_pred_rf = rf.predict(X_test[vars_model])
y_val_rf = rf.predict(df_val[vars_model])

# Performance
rf_result_oos = ml_error('Random Forest Regressor OOS', y_test, y_pred_rf)
rf_result_oos

In [ ]:
rf_result_oot = ml_error('Random Forest Regressor OOT', df_val.Weekly_Sales, y_val_rf)
rf_result_oot

In [ ]:
aux_compara = pd.concat([aux_compara, rf_result_oos],axis = 0)
aux_compara = pd.concat([aux_compara, rf_result_oot],axis = 0)


aux_compara

## Gradient Boosting - XGB

* Da mesma forma que o Random Forest, tive que rodar a etapa de otimização dos hiperparâmetros na minha máquina local.
* Como os modelos xgb demoram ainda mais para rodar e, consequentemente, testar os parâmetros, utilizei um método não-exaustivo para acelerar o processo de otimização dos parâmetros.
* O código utilizado está comentado aqui.

In [ ]:
# from skopt.space import Real, Integer
# from skopt.utils import use_named_args

# n_features = X_train[vars_model].shape[1]

# XGB_opt = GradientBoostingClassifier(n_estimators=100, random_state=0, criterion = 'mse')

# space  = [Integer(5,15, name='max_depth')
#          ,Real(10**-5, 0.1, "log-uniform", name='learning_rate')
#          ,Real(0.25, 0.75,'log-unifom', name = 'subsample')
#          ,Integer(2, 5, name='min_samples_split')
#          ,Integer(1, 4, name='min_samples_leaf')]


#@use_named_args(space)
#def objective(**params):
#   XGB_opt.set_params(**params)

#    return -np.mean(cross_val_score(XGB_opt, X_train[vars_model], y_train, cv=5, n_jobs=-1,
#                                    scoring="neg_root_mean_squared_error"))

In [ ]:
#from skopt import gp_minimize
#res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)

#"Best score=%.4f" % res_gp.fun

In [ ]:
#print("""Best parameters:
#- max_depth=%d
#- learning_rate=%.6f
#- subsample=%.2f
#- min_samples_split=%d
#- min_samples_leaf=%d""" % (res_gp.x[0], res_gp.x[1],
#                            res_gp.x[2], res_gp.x[3],
#                            res_gp.x[4]))

In [ ]:
# model
xgb_model = GradientBoostingRegressor(n_estimators= 100
                                     ,learning_rate=0.1
                                     ,max_depth=15
                                     ,subsample=0.75)

xgb_model.fit(X_train[vars_model], y_train)

In [ ]:
# prediction
y_pred_xgb = xgb_model.predict(X_test[vars_model])
y_val_xgb = xgb_model.predict(df_val[vars_model])

# performance
xgb_result_oos = ml_error('XGBoost Regressor OOS', y_test, y_pred_xgb)
xgb_result_oos

In [ ]:
# performance
xgb_result_oot = ml_error('XGBoost Regressor OOT', df_val.Weekly_Sales, y_val_xgb)
xgb_result_oot

In [ ]:
aux_compara = pd.concat([aux_compara, xgb_result_oos],axis = 0)
aux_compara = pd.concat([aux_compara, xgb_result_oot],axis = 0)


aux_compara

## Análise sobre os modelos testados

* Como esperado, os modelos Random Forest (RF) e Gradient Boost (XGB) demonstraram desenpenho muito superior ao modelo Baseline e também em relação à regressão com regularização.
* Os modelos RF e XGB mostraram desempenho semelhante. No entanto, dada a maior compledixade do XGB, faz sentido seguir com o Random Forest.

# Análise sobre o modelo final

* Nesta etapa vale explorar as variáveis com um pouco mais de cuidado. 
* Baseado no valor da importâcia, escolhi entre as variáveis absolutas e relativas, alem de remover as variáveis com importância menor que 0,001.


## Importância das Variáveis

In [ ]:
Import_vars_rf = pd.DataFrame({'Var':vars_model,
                               'Importancia': rf.feature_importances_})

#Ordenação das variáveis pela importância
Import_vars_rf.sort_values('Importancia',ascending = False, inplace = True)
Import_vars_rf.reset_index(drop=True,inplace = True)

In [ ]:
Import_vars_rf

In [ ]:
plt.rcdefaults()
fig, ax = plt.subplots(figsize = (6,8))

# Example data
var = Import_vars_rf['Var']
y_pos = np.arange(len(var))
importance = Import_vars_rf['Importancia']

ax.barh(y_pos, importance, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(var)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('importance')
ax.set_ylabel('variaveis')
ax.set_title('Importância Variáveis')

plt.show()

In [ ]:
Import_vars_rf.Var.unique()

In [ ]:
vars_model_opt = ['Dept', 
                  'Size', 
                  'Store', 
                  'week_of_year', 
                  'CPI', 
                  #'relative_CPI',
                  'Type_A', 
                  'Type_B', 
                  'day', 
                  'month', 
                  'Temperature', 
                  'Unemployment',
                  #'relative_Unemployment', 
                  #'relative_Temperature', 
                  'MarkDown3',
                  #'Type_C', 
                  'Fuel_Price', 
                  #'relative_Fuel_Price', 
                  'MarkDown4',
                  'MarkDown5', 
                  'IsHoliday_True'
                  #,'MarkDown1', 'MarkDown2', 'year'
                ]

In [ ]:
# model
rf = RandomForestRegressor(n_estimators = 100
                           ,max_depth = None
                           ,max_features = 0.75)

rf.fit(X_train[vars_model_opt], y_train)

In [ ]:
# prediction
y_pred_rf_opt = rf.predict(X_test[vars_model_opt])
y_val_rf_opt = rf.predict(df_val[vars_model_opt])

# Performance
rf_result_opt_oos = ml_error('Random Forest Regressor Opt OOS', y_test, y_pred_rf_opt)
rf_result_opt_oos

In [ ]:
rf_result_opt_oot = ml_error('Random Forest Regressor Opt OOT', df_val.Weekly_Sales, y_val_rf_opt)
rf_result_opt_oot

In [ ]:
aux_compara = pd.concat([aux_compara, rf_result_opt_oos],axis = 0)
aux_compara = pd.concat([aux_compara, rf_result_opt_oot],axis = 0)


aux_compara

## Baseline - Regressão Linear com variáveis selecionadas pelo Random Forest

* Aqui removi as varáveis correlacionadas, optando pela variável com maior importância dada pelo modelo anterior.

In [ ]:
LR_opt = LinearRegression()

LR_opt.fit(X_train[vars_model_opt],y_train)


In [ ]:
y_pred_opt = LR_opt.predict(X_test[vars_model_opt])
y_val_opt = LR_opt.predict(df_val[vars_model_opt])

#performance oos
lr_result_opt_oos = ml_error('Linear Regression - OPT - OOS', y_test, y_pred_opt)
lr_result_opt_oos

In [ ]:
# performance oot
lr_result_opt_oot = ml_error('Linear Regression - OPT - OOT', df_val.Weekly_Sales, y_val_opt)
lr_result_opt_oot

In [ ]:
aux_compara = pd.concat([aux_compara, lr_result_opt_oos],axis = 0)
aux_compara = pd.concat([aux_compara, lr_result_opt_oot],axis = 0)


aux_compara

## Regressão Linear - l1 (Lasso) - Com variáveis selecionadas pelo Random Forest

In [ ]:
# model
lrr_rff_opt = Lasso(alpha = 0.01)
lrr_rff_opt.fit(X_train[vars_model_opt], y_train)

In [ ]:
# prediction
y_pred_lrr_opt = lrr_rff_opt.predict(X_test[vars_model_opt])
y_val_lrr_opt = lrr_rff_opt.predict(df_val[vars_model_opt])

#performance
lrr_result_opt_oos = ml_error('Linear Regression - Lasso - OPT - OOS', y_test, y_pred_lrr_opt)
lrr_result_opt_oos

In [ ]:
lrr_result_opt_oot = ml_error('Linear Regression - Lasso - OPT - OOT', df_val.Weekly_Sales, y_val_lrr_opt)
lrr_result_opt_oot

In [ ]:
aux_compara = pd.concat([aux_compara, lrr_result_opt_oos],axis = 0)
aux_compara = pd.concat([aux_compara, lrr_result_opt_oot],axis = 0)


aux_compara

# Comparação entre os modelos utilizando a métrica proposta pela competição

* **Métrica:** Weighted Mean Absolute Error (WMAE)
* A WMAE é semelhante ao Mean Absolute Error (MAE), que foi avaliada no desenvolvimento. 
* A grande diferença aqui é que as semanas que contem feriados tem um peso maior na avaliação do modelo

## Avaliação do erro no conjunto de teste (OOS)

In [ ]:
X_test.head()

In [ ]:
df_erros = pd.DataFrame({'is_holiday':X_test.IsHoliday_True
                         ,'y_test':y_test
                         ,'y_pred_LR':y_pred_opt
                         ,'y_pred_Lasso':y_pred_lrr_opt
                         ,'y_pred_RF':y_pred_rf_opt
                         ,'y_pred_XGB':y_pred_xgb})

In [ ]:
df_erros.head()

In [ ]:
df_erros['w'] = df_erros['is_holiday'].apply(lambda x: 5 if x == 1 else 1)

df_erros.loc[:,'prod_LR'] = df_erros.loc[:,'y_test'] - df_erros.loc[:,'y_pred_LR']
df_erros.loc[:,'prod_Lasso'] = df_erros.loc[:,'y_test'] - df_erros.loc[:,'y_pred_Lasso']
df_erros.loc[:,'prod_RF'] = df_erros.loc[:,'y_test'] - df_erros.loc[:,'y_pred_RF']
df_erros.loc[:,'prod_XGB'] = df_erros.loc[:,'y_test'] - df_erros.loc[:,'y_pred_XGB']

df_erros.loc[:,'w_prod_LR'] = df_erros.loc[:,'prod_LR'] * df_erros.loc[:,'w']
df_erros.loc[:,'w_prod_Lasso'] = df_erros.loc[:,'prod_Lasso'] * df_erros.loc[:,'w']
df_erros.loc[:,'w_prod_RF'] = df_erros.loc[:,'prod_RF'] * df_erros.loc[:,'w']
df_erros.loc[:,'w_prod_XGB'] = df_erros.loc[:,'prod_XGB'] * df_erros.loc[:,'w']

erro_LR = np.abs(df_erros.w_prod_LR.sum() / df_erros.w.sum()).round(2)
erro_Lasso = np.abs(df_erros.w_prod_Lasso.sum() / df_erros.w.sum()).round(2)
erro_RF = np.abs(df_erros.w_prod_RF.sum() / df_erros.w.sum()).round(2)
erro_XGB = np.abs(df_erros.w_prod_XGB.sum() / df_erros.w.sum()).round(2)

In [ ]:
print('erro_LR', erro_LR)
print('erro_Lasso', erro_Lasso)
print('erro_RF', erro_RF)
print('erro_XGB', erro_XGB)

## Avaliação do erro no conjunto de validação (OOT)

In [ ]:
df_val.head()

In [ ]:
df_erros_val = pd.DataFrame({'is_holiday':df_val.IsHoliday_True
                             ,'y_val':df_val.Weekly_Sales
                             ,'y_val_LR':y_val_opt
                             ,'y_val_Lasso':y_val_lrr_opt
                             ,'y_val_RF':y_val_rf_opt
                             ,'y_val_XGB':y_val_xgb})

df_erros_val['w'] = df_erros_val['is_holiday'].apply(lambda x: 5 if x == 1 else 1)

df_erros_val.loc[:,'prod_LR'] = df_erros_val.loc[:,'y_val'] - df_erros_val.loc[:,'y_val_LR']
df_erros_val.loc[:,'prod_Lasso'] = df_erros_val.loc[:,'y_val'] - df_erros_val.loc[:,'y_val_Lasso']
df_erros_val.loc[:,'prod_RF'] = df_erros_val.loc[:,'y_val'] - df_erros_val.loc[:,'y_val_RF']
df_erros_val.loc[:,'prod_XGB'] = df_erros_val.loc[:,'y_val'] - df_erros_val.loc[:,'y_val_XGB']

df_erros_val.loc[:,'w_prod_LR'] = df_erros_val.loc[:,'prod_LR'] * df_erros_val.loc[:,'w']
df_erros_val.loc[:,'w_prod_Lasso'] = df_erros_val.loc[:,'prod_Lasso'] * df_erros_val.loc[:,'w']
df_erros_val.loc[:,'w_prod_RF'] = df_erros_val.loc[:,'prod_RF'] * df_erros_val.loc[:,'w']
df_erros_val.loc[:,'w_prod_XGB'] = df_erros_val.loc[:,'prod_XGB'] * df_erros_val.loc[:,'w']



erro_LR_val = np.abs(df_erros_val.w_prod_LR.sum() / df_erros_val.w.sum()).round(2)
erro_Lasso_val = np.abs(df_erros_val.w_prod_Lasso.sum() / df_erros_val.w.sum()).round(2)
erro_RF_val = np.abs(df_erros_val.w_prod_RF.sum() / df_erros_val.w.sum()).round(2)
erro_XGB_val = np.abs(df_erros_val.w_prod_XGB.sum() / df_erros_val.w.sum()).round(2)

In [ ]:
df_erros_val.head()

In [ ]:
df_err_consolid = pd.DataFrame({'Modelo': ['Baseline - Reg. Lin.', 'Baseline - Reg. Lin.', 'Reg. Lin. - Lasso', 'Reg. Lin. - Lasso', 'Random Forest', 'Random Forest', 'Gradient Boost', 'Gradient Boost']
                               ,'Etapa': ['Teste - OOS', 'Validação - OOT', 'Teste - OOS', 'Validação - OOT', 'Teste - OOS', 'Validação - OOT', 'Teste - OOS', 'Validação - OOT']
                               ,'Erro':[erro_LR, erro_LR_val, erro_Lasso, erro_Lasso_val, erro_RF, erro_RF_val, erro_XGB, erro_XGB_val]})

In [ ]:
plt.figure()
plt.title ('Erro Absoluto Ponderado pelos Feriados por Modelo')
fig = sns.barplot(x = 'Modelo'
                  ,y = 'Erro'
                  ,data = df_err_consolid
                  ,hue = 'Etapa')

In [ ]:
print('erro_LR_val:', erro_LR_val)
print('erro_Lasso_val:', erro_Lasso_val)
print('erro_RF_val:', erro_RF_val)
print('erro_XGB_val:', erro_XGB_val)

In [ ]:
df_err_consolid

# Análise das predições do modelo para o volume de vendas no período analisado

* Aqui podemos avaliar como o modelo se comportou por loja, por departamento e por período.
* Isto nos permite avaliar se é necessário ter um modelo separado por loja ou por departamento.
* Também podemos verificar se o modelo consegue prever os picos de vendas que ocorrem em feriados específicos. 

In [ ]:
df_result = pd.DataFrame({'Store':viz_result_test['Store']
                          ,'Dept':viz_result_test['Dept']
                          ,'Date':viz_result_test['Date']
                          ,'Year':viz_result_test['year']
                          ,'Week':viz_result_test['week_of_year']
                          ,'Weekly Sales': y_test
                          ,'Weekly Sales Estimated': y_pred_rf_opt})

df_plot_store = pd.melt(df_result, id_vars = "Store", value_vars = ["Weekly Sales", 'Weekly Sales Estimated'], var_name = 'real_vs_estimated', value_name="Sales")
df_plot_dept = pd.melt(df_result, id_vars = "Dept", value_vars = ["Weekly Sales", 'Weekly Sales Estimated'], var_name = 'real_vs_estimated', value_name="Sales")

In [ ]:
df_plot_store.head()

In [ ]:
plt.figure(figsize = (20,6))
plt.title ('Avaliação das vendas estimadas em relação ao real separado por loja')
fig = sns.boxplot(x = 'Store'
                  ,y = 'Sales'
                  ,data = df_plot_store[['Store','Sales','real_vs_estimated']]
                  ,showfliers = False
                  ,hue = 'real_vs_estimated')

In [ ]:
df_plot_dept.head()

In [ ]:
plt.figure(figsize = (20,6))
plt.title ('Avaliação das vendas estimadas em relação ao real separado por loja')
fig = sns.boxplot(x = 'Dept'
                  ,y = 'Sales'
                  ,data = df_plot_dept[['Dept','Sales','real_vs_estimated']]
                  ,showfliers = False
                  ,hue = 'real_vs_estimated')

### Volume de vendas por semana ao longo do ano

* Por fim, podemos avaliar o volume de vendas por semana, se existe alguma sazonalidade e/ou influência dos feriados que possa nos direcionar na construção de novas variáveis e no desenvolvimento do  modelo.

In [ ]:
real_weekly_sales_2010 = df_result[df_result.Year==2010].groupby('Week')['Weekly Sales'].mean()
real_weekly_sales_2011 = df_result[df_result.Year==2011].groupby('Week')['Weekly Sales'].mean()
real_weekly_sales_2012 = df_result[df_result.Year==2012].groupby('Week')['Weekly Sales'].mean()

estimated_weekly_sales_2010 = df_result[df_result.Year==2010].groupby('Week')['Weekly Sales Estimated'].mean()
estimated_weekly_sales_2011 = df_result[df_result.Year==2011].groupby('Week')['Weekly Sales Estimated'].mean()
estimated_weekly_sales_2012 = df_result[df_result.Year==2012].groupby('Week')['Weekly Sales Estimated'].mean()


In [ ]:
plt.figure(figsize=(20,6))
# plt.plot(weekly_sales_2010.index, weekly_sales_2010.values)
# plt.plot(weekly_sales_2011.index, weekly_sales_2011.values)
plt.plot(real_weekly_sales_2010.index, real_weekly_sales_2010.values)
plt.plot(estimated_weekly_sales_2010.index, estimated_weekly_sales_2010.values)

plt.xticks(np.arange(1, 53, step=1), fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('Week of Year', fontsize=16, labelpad=20)
plt.ylabel('Sales', fontsize=20, labelpad=20)

plt.title("Volume de vendas por semana de 2010, real versus estimado", fontsize=16)
plt.legend(['Real', 'Estimado'], fontsize=14);

In [ ]:
plt.figure(figsize=(20,6))
# plt.plot(weekly_sales_2010.index, weekly_sales_2010.values)
# plt.plot(weekly_sales_2011.index, weekly_sales_2011.values)
plt.plot(real_weekly_sales_2011.index, real_weekly_sales_2011.values)
plt.plot(estimated_weekly_sales_2011.index, estimated_weekly_sales_2011.values)

plt.xticks(np.arange(1, 53, step=1), fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('Week of Year', fontsize=16, labelpad=20)
plt.ylabel('Sales', fontsize=20, labelpad=20)

plt.title("Volume de vendas por semana de 2011, real versus estimado", fontsize=16)
plt.legend(['Real', 'Estimado'], fontsize=14);

In [ ]:
plt.figure(figsize=(20,6))
# plt.plot(weekly_sales_2010.index, weekly_sales_2010.values)
# plt.plot(weekly_sales_2011.index, weekly_sales_2011.values)
plt.plot(real_weekly_sales_2012.index, real_weekly_sales_2012.values)
plt.plot(estimated_weekly_sales_2012.index, estimated_weekly_sales_2012.values)

plt.xticks(np.arange(1, 53, step=1), fontsize=16)
plt.yticks( fontsize=16)
plt.xlabel('Week of Year', fontsize=16, labelpad=20)
plt.ylabel('Sales', fontsize=20, labelpad=20)

plt.title("Volume de vendas por semana de 2012, real versus estimado", fontsize=16)
plt.legend(['Real', 'Estimado'], fontsize=14);

# Análise dos resultados sob a perspectiva do WMAE

* A avaliação do desempenho do modelo sob a perspectiva do erro médio ponderado pelos feriados trouxe uma visão bem diferente das outras métricas analisadas no desenvolvimento. 
* Quando olhamos para os valores de erro dentro do mesmo período em que o modelo foi desenvolvido, observamos valores semelhantes de erro.
* No entanto, o desempenho do modelo em um período diferente do que foi treinado foi muito inferior para as técinas de regressão. 
* É importante salientar aqui que, por razão do tempo, só avaliei um mês, que pode ser um mês atípico, mesmo possibilitando a análise de 4 semanas distintas. 
* Ainda assim, tanto o random forest quanto o gradient boost foram superiores.
* Entre os dois últimos, temos valores próximos de desempenho. O Randon Forest (RF) mostrou valores maiores de erro na base de teste e valores menores no dataset de validação fora do tempo.
* Neste caso, como os valores são próximos, o RF mostrou melhor desempenho fora do período de desenvolvimento e é uma técnica mais simples, isto é, mais barata do ponto de vista computacional, escolhi seguir com o Random Forest para a análise da base de Teste.
* Quando comparamos os valores reais com os previstos, podemos perceber qu eo modelo funcionou bem para a previsão de todas as lojas, com pequenas variações.
* O modelo também mostrou boa performance para todos os departamentos, com exceção do 39.
* Por fim, quando avaliamos as previsões semana a semana, podemos observar que o modelo consegue prever de forma eficiente as vendas, inclusive nos períodos de feriados

# Armazenando modelo final

* Os resultados acima mostraram que o melhor modelo foi o Random Forest.
* Assim, aqui armazenamos o modelo em um pickle para poder usá-lo na base de teste, de forma que este processo seja reprodutível em produção.

In [ ]:
regressor_Pkl = pickle.dumps(rf)

# Aplicando o modelo na base de teste

* Nesta etapa, podemos aproveitar para avaliar o desempenho do modelo e também para garantir que o modelo, bem como as transformações necessárias nas variáveis, podem ser aplicadas em uma nova base.
* Para esta etapa, podemos utilizar o modelo e a função de normalização (scaler) que foram armazenadas em arquivos pickle anteriormente, durante o desenvolvimento.

In [ ]:
test.head()


* Aqui achei interessante construir o código de forma que seria possível utilizar esta célula como um todo para implantação do modelo em produção, uma vez que ele traz todas as bibliotecas, funções e listas necessárias para escoragem da base pelo modelo.

In [ ]:
# Para transformar esta célula em um arquivo .py 

# %%writefile Forecasting_Walmart_PyScore.py

# ***********************************************************
# *********** Imports ***************************************
# ***********************************************************

import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor

# ***********************************************************
# ******* Funções Utilizadas ********************************
# ***********************************************************

def split_date(df,date):

    '''
    Transforma o campo indicado na entrada em data e extrai valores que podem ser relevantes, como ano, mês e dia. 
    A saida da função concatena esses novos campos no dataframe indicado na entrada
    '''
    
    df['dt_ref'] = pd.to_datetime(df[date])
    df['year'] = df.dt_ref.dt.year
    df['month'] = df.dt_ref.dt.month
    df['day'] = df.dt_ref.dt.day
    df['week_of_year'] = df.dt_ref.dt.isocalendar().week
    df['period_month'] = df_data.dt_ref.dt.to_period('M')
    
def variaveis_relativas(df, lista_vars, var_group):
    
    '''
    O objetivo desta função é criar 'valores relativos', que indicam o quanto o valor mais atual de distancia do valor médio de um período de interesse.
    Aqui foi testado o valor da variável em relação à média mensal, que é indicada pela variável 'var_group'.
    Entradas: DataFrame com todas as informações necessárias para calcular os valores consolidados, lista dos campos que serão transformados e a variável sobre a qual os valores serão agrupados
    Saídas: A função devolve o mesmo DataFrame, com as novas variáveis concatenadas 
    '''
    
    # Calculando valores médios para servir de referencial 
    aux_mean = pd.DataFrame()
    aux_names = [var_group]
    
    for var in lista_vars:
        #print(var)
        aux_mean[var] = df[[var_group,var]].groupby([var_group]).mean()
        aux_name_2 = 'mean_' + var
        aux_names.append(aux_name_2)

    aux_mean.reset_index(inplace = True)

    # Ajustando o nome dos campos
    aux_mean.columns = aux_names

    df = df.merge(aux_mean
                  ,on = [var_group]
                  ,how = 'inner')
    
    #Incluindo as variáveis no dataframe
    for var in lista_vars:
        var_name = 'relative_' + var
        mean_var = 'mean_' + var
        df[var_name] = (df[var] + 0.0001)/df[mean_var]
        
    return df

# ***********************************************************
# ******* Listas Importantes ********************************
# ***********************************************************


# Variáveis para o modelo
input_cols = ['Store', 'Dept', 'Weekly_Sales', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size', 'year',
       'month', 'day', 'week_of_year']

# Variáveis numéricas
var_num = ['Store', 'Dept', 'Size','Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'year', 'month', 'day',
       'week_of_year']

var_num_2 = var_num + ['relative_Temperature'
                       ,'relative_Fuel_Price'
                       ,'relative_CPI'
                       ,'relative_Unemployment']


# Variáveis categóricas
var_cat = ['IsHoliday', 'Type']

# Variável alvo
target_col = 'Weekly_Sales'

vars_model = ['Store',
             'Dept',
             'Size',
             'Temperature',
             'Fuel_Price',
             'MarkDown1',
             'MarkDown2',
             'MarkDown3',
             'MarkDown4',
             'MarkDown5',
             'CPI',
             'Unemployment',
             'year',
             'month',
             'day',
             'week_of_year',
             'relative_Temperature',
             'relative_Fuel_Price',
             'relative_CPI',
             'relative_Unemployment',
             'IsHoliday_True',
             'Type_A',
             'Type_B',
             'Type_C']

vars_model_opt = ['Dept', 
                  'Size', 
                  'Store', 
                  'week_of_year', 
                  'CPI', 
                  'Type_A', 
                  'Type_B', 
                  'day', 
                  'month', 
                  'Temperature', 
                  'Unemployment',
                  'MarkDown3',
                  'Fuel_Price',  
                  'MarkDown4',
                  'MarkDown5', 
                  'IsHoliday_True']

# ***********************************************************
# ******* Carregando os Datasetes ***************************
# ***********************************************************

# Carregando as bases em dataframes panda
features = pd.read_csv("./walmart-recruiting-store-sales-forecasting/features.csv")
stores = pd.read_csv("../input/walmart-recruiting-store-sales-forecasting/stores.csv")
test = pd.read_csv("./walmart-recruiting-store-sales-forecasting/test.csv")

# Combinando o dataset de teste com as outras informações disponíveis para conseguir geraras previsões
df_test = test.merge(features
                     ,on = ['Store','Date','IsHoliday']
                     ,how = 'inner').merge(stores
                                           ,on = ['Store']
                                           ,how = 'inner')

# ***********************************************************
# ******* Preparação da base ********************************
# ***********************************************************

# Incluindo formas diferentes de imputar valores para os campos missing nas variáveis

# Aqui escolhi inputar o valor médio das variáveis para todo o dataset. 
# Em produção, este valor deveria ser fixo, pois qualquer variação no comportamento da população pode influenciar no comportamento da variável
mean_list = ['Temperature','Fuel_Price','CPI','Unemployment']

# Aqui mantive a ideia inicial, utilizada para o dataset de treino, substituindo por zero.
markdowns = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']

for var in mean_list:
    df_test[var] = df_test[var].fillna(df_test[var].mean())

for var in markdowns:
    df_test[var] = df_test[var].fillna(0)
    
# Aplicando as funções para construção, primeiro das variáveis relacionadas com as datas e períodos de interesse, e depois para construção das variáveis relativas.
split_date(df_test, 'Date')
df_test = variaveis_relativas(df_test, mean_list, 'month')

# Normalizando as variáveis contínuas com o scaler que foi ajustado com o dataset de treino e armazenado no pickle.
scaler = pickle.loads(scaler_Pkl) 
df_test[var_num_2] = scaler.transform(df_test[var_num_2])

# Inclusão das variáveis categóricas, já binarizadas, na lista geral de variáveis.
# Como são poucas variáveis, decidi fazer o ajuste manual.
df_test['IsHoliday_True'] = np.where(df_test.IsHoliday == True, 1,0)
df_test['Type_A'] = np.where(df_test.Type == 'A', 1,0)
df_test['Type_B'] = np.where(df_test.Type == 'B', 1,0)
df_test['Type_C'] = np.where(df_test.Type == 'C', 1,0)

# A definição desta lista, assim como as outras no início da célula, é redundante no notebook, mas se considerarmos que a idéia desta célula é ter um código que poderia ser levado para produção, estas definições são importantes.
vars_model = var_num_2 + ['IsHoliday_True', 'Type_A', 'Type_B', 'Type_C']

# ***********************************************************
# ********* Escorando a base ********************************
# ***********************************************************


rf_pickle = pickle.loads(regressor_Pkl) 

# Escoragem da população na base teste
y_pred_rf_test = rf.predict(df_test[vars_model_opt])